<a href="https://colab.research.google.com/github/RenilGarala/fraud-detection-uniforms/blob/main/fraud_detection_uniforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe
!pip install tensorflow
!pip install opencv-python==4.8.0.76
!pip install matplotlib seaborn
!pip install scikit-learn pandas
!pip install tqdm requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88


In [ ]:
!pip install seaborn
!pip install pillow

In [ ]:
import tensorflow as tf
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from google.colab import drive, files
import zipfile
import shutil

In [ ]:
# Mount Google Drive
drive.mount('/content/MyDrive')

# Create proper directory structure
base_dir = '/content/MyDrive/'
os.makedirs(f'{base_dir}/uniform/train', exist_ok=True)
os.makedirs(f'{base_dir}/uniform/test', exist_ok=True)

print("Directory structure created!")

Directory structure created!


In [ ]:

import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil

class DataPreprocessor:
    def __init__(self, data_path="Uniform"):
        self.data_path = data_path
        self.image_size = (224, 224)  # EfficientNet input size

    def load_and_preprocess_images(self):
        """Load and preprocess all images from train/test folders"""

        train_images = []
        test_images = []
        train_labels = []
        test_labels = []

        # Process training data
        train_path = os.path.join(self.data_path, "train")
        if os.path.exists(train_path):
            for filename in os.listdir(train_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(train_path, filename)
                    img = self.preprocess_image(img_path)
                    if img is not None:
                        train_images.append(img)
                        # Determine label based on filename or folder structure
                        label = self.determine_label(filename)
                        train_labels.append(label)

        # Process test data
        test_path = os.path.join(self.data_path, "test")
        if os.path.exists(test_path):
            for filename in os.listdir(test_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(test_path, filename)
                    img = self.preprocess_image(img_path)
                    if img is not None:
                        test_images.append(img)
                        label = self.determine_label(filename)
                        test_labels.append(label)

        return np.array(train_images), np.array(train_labels), np.array(test_images), np.array(test_labels)

    def preprocess_image(self, img_path):
        """Preprocess single image"""
        try:
            # Read image
            img = cv2.imread(img_path)
            if img is None:
                return None

            # Convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Resize to target size
            img = cv2.resize(img, self.image_size)

            # Normalize pixel values
            img = img.astype(np.float32) / 255.0

            return img
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            return None

    def determine_label(self, filename):
        """Determine label based on filename - you'll need to adapt this"""
        # This is a simple example - modify based on your naming convention
        filename_lower = filename.lower()

        if any(keyword in filename_lower for keyword in ['police', 'cop', 'officer', 'uniform']):
            return 1  # Uniform
        else:
            return 0  # Non-uniform

    def create_organized_dataset(self):
        """Create properly organized dataset structure"""

        # Create organized structure
        organized_path = "organized_dataset"
        os.makedirs(f"{organized_path}/train/uniform", exist_ok=True)
        os.makedirs(f"{organized_path}/train/non_uniform", exist_ok=True)
        os.makedirs(f"{organized_path}/test/uniform", exist_ok=True)
        os.makedirs(f"{organized_path}/test/non_uniform", exist_ok=True)

        print("Organized dataset structure created!")
        print("Please manually organize your images into:")
        print("- organized_dataset/train/uniform/ (for uniform images)")
        print("- organized_dataset/train/non_uniform/ (for civilian clothes)")
        print("- organized_dataset/test/uniform/ (for test uniform images)")
        print("- organized_dataset/test/non_uniform/ (for test civilian images)")

        return organized_path

    def visualize_samples(self, images, labels, num_samples=8):
        """Visualize sample images"""
        plt.figure(figsize=(15, 10))

        for i in range(min(num_samples, len(images))):
            plt.subplot(2, 4, i+1)
            plt.imshow(images[i])
            label_text = "Uniform" if labels[i] == 1 else "Non-uniform"
            plt.title(f"Label: {label_text}")
            plt.axis('off')

        plt.tight_layout()
        plt.savefig('results/sample_images.png', dpi=300, bbox_inches='tight')
        plt.show()

# Usage example
preprocessor = DataPreprocessor()

# Create organized structure
organized_path = preprocessor.create_organized_dataset()

print("Data preprocessing setup completed!")
print("\nNext steps:")
print("1. Organize your images in the created folder structure")
print("2. Run the image segmentation script")
print("3. Train the classification model")

Organized dataset structure created!
Please manually organize your images into:
- organized_dataset/train/uniform/ (for uniform images)
- organized_dataset/train/non_uniform/ (for civilian clothes)
- organized_dataset/test/uniform/ (for test uniform images)
- organized_dataset/test/non_uniform/ (for test civilian images)
Data preprocessing setup completed!

Next steps:
1. Organize your images in the created folder structure
2. Run the image segmentation script
3. Train the classification model


In [ ]:

import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

class UniformSegmenter:
    def __init__(self):
        # Initialize MediaPipe solutions
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils

        # Initialize models
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=1)
        self.pose = self.mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

    def segment_person(self, image):
        """Segment person from background using MediaPipe"""

        # Convert image to RGB if needed
        if len(image.shape) == 3:
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if image.shape[2] == 3 else image
        else:
            rgb_image = image

        # Get segmentation mask
        results = self.selfie_segmentation.process(rgb_image)

        # Create binary mask
        mask = results.segmentation_mask
        mask_binary = (mask > 0.5).astype(np.uint8)

        # Apply mask to original image
        segmented_image = rgb_image * mask_binary[:, :, np.newaxis]

        return segmented_image, mask_binary

    def detect_pose_keypoints(self, image):
        """Detect pose keypoints to identify uniform regions"""

        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else image
        results = self.pose.process(rgb_image)

        keypoints = []
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                keypoints.append([landmark.x, landmark.y, landmark.z])

        return np.array(keypoints), results

    def extract_uniform_regions(self, image, keypoints):
        """Extract specific uniform regions based on pose keypoints"""

        if len(keypoints) == 0:
            return []

        h, w = image.shape[:2]
        regions = []

        # Define regions of interest for uniform detection
        # Torso region (chest area)
        if len(keypoints) >= 12:  # Ensure we have shoulder landmarks
            left_shoulder = keypoints[11]  # Left shoulder
            right_shoulder = keypoints[12]  # Right shoulder

            # Calculate torso bounding box
            chest_x1 = int(min(left_shoulder[0], right_shoulder[0]) * w) - 50
            chest_x2 = int(max(left_shoulder[0], right_shoulder[0]) * w) + 50
            chest_y1 = int(min(left_shoulder[1], right_shoulder[1]) * h)
            chest_y2 = chest_y1 + 150  # Approximate chest height

            # Ensure coordinates are within image bounds
            chest_x1 = max(0, chest_x1)
            chest_x2 = min(w, chest_x2)
            chest_y1 = max(0, chest_y1)
            chest_y2 = min(h, chest_y2)

            if chest_x2 > chest_x1 and chest_y2 > chest_y1:
                chest_region = image[chest_y1:chest_y2, chest_x1:chest_x2]
                regions.append(('chest', chest_region, (chest_x1, chest_y1, chest_x2, chest_y2)))

        # Head region (for caps/hats)
        if len(keypoints) >= 1:
            nose = keypoints[0]  # Nose landmark

            head_x1 = int(nose[0] * w) - 80
            head_x2 = int(nose[0] * w) + 80
            head_y1 = int(nose[1] * h) - 120
            head_y2 = int(nose[1] * h) - 20

            head_x1 = max(0, head_x1)
            head_x2 = min(w, head_x2)
            head_y1 = max(0, head_y1)
            head_y2 = min(h, head_y2)

            if head_x2 > head_x1 and head_y2 > head_y1:
                head_region = image[head_y1:head_y2, head_x1:head_x2]
                regions.append(('head', head_region, (head_x1, head_y1, head_x2, head_y2)))

        return regions

    def process_image(self, image_path):
        """Complete processing pipeline for an image"""

        # Read image
        image = cv2.imread(image_path)
        if image is None:
            return None, None, []

        # Step 1: Segment person
        segmented_image, mask = self.segment_person(image)

        # Step 2: Detect pose keypoints
        keypoints, pose_results = self.detect_pose_keypoints(image)

        # Step 3: Extract uniform regions
        regions = self.extract_uniform_regions(image, keypoints)

        return segmented_image, mask, regions

    def visualize_segmentation(self, original_image, segmented_image, mask, regions):
        """Visualize segmentation results"""

        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        # Original image
        axes[0, 0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title('Original Image')
        axes[0, 0].axis('off')

        # Segmented image
        axes[0, 1].imshow(segmented_image)
        axes[0, 1].set_title('Segmented Person')
        axes[0, 1].axis('off')

        # Segmentation mask
        axes[0, 2].imshow(mask, cmap='gray')
        axes[0, 2].set_title('Segmentation Mask')
        axes[0, 2].axis('off')

        # Show extracted regions
        for i, (region_name, region_img, bbox) in enumerate(regions[:3]):
            if i < 3:
                if len(region_img.shape) == 3:
                    axes[1, i].imshow(cv2.cvtColor(region_img, cv2.COLOR_BGR2RGB))
                else:
                    axes[1, i].imshow(region_img, cmap='gray')
                axes[1, i].set_title(f'{region_name} Region')
                axes[1, i].axis('off')

        # Hide unused subplots
        for i in range(len(regions), 3):
            axes[1, i].axis('off')

        plt.tight_layout()
        plt.savefig('results/segmentation_results.png', dpi=300, bbox_inches='tight')
        plt.show()

# Usage example
def process_sample_images():
    segmenter = UniformSegmenter()

    # Process images from your dataset
    data_path = "Uniform/train"
    if os.path.exists(data_path):
        sample_images = [f for f in os.listdir(data_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for i, img_name in enumerate(sample_images[:3]):  # Process first 3 images as examples
            img_path = os.path.join(data_path, img_name)
            print(f"Processing: {img_name}")

            original_image = cv2.imread(img_path)
            segmented_image, mask, regions = segmenter.process_image(img_path)

            if segmented_image is not None:
                print(f"Found {len(regions)} regions: {[r[0] for r in regions]}")
                segmenter.visualize_segmentation(original_image, segmented_image, mask, regions)
            else:
                print(f"Failed to process {img_name}")
    else:
        print(f"Path {data_path} not found. Please upload your images first.")

# Initialize segmenter
segmenter = UniformSegmenter()
print("Image segmentation module ready!")
print("Run process_sample_images() to test segmentation on your data")

Image segmentation module ready!
Run process_sample_images() to test segmentation on your data


In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os

class UniformClassifier:
    def __init__(self, input_shape=(224, 224, 3), num_classes=2):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = None
        self.history = None

    def create_model(self):
        """Create EfficientNet-Lite4 based model"""

        # Load pre-trained EfficientNetB4 (closest to EfficientNet-Lite4)
        base_model = EfficientNetB4(
            weights='imagenet',
            include_top=False,
            input_shape=self.input_shape
        )

        # Freeze base model layers initially
        base_model.trainable = False

        # Add custom classification head
        inputs = keras.Input(shape=self.input_shape)

        # Preprocessing
        x = tf.cast(inputs, tf.float32)
        x = tf.keras.applications.efficientnet.preprocess_input(x)

        # Base model
        x = base_model(x, training=False)

        # Classification head
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.2)(x)

        # Output layer
        outputs = layers.Dense(self.num_classes, activation='softmax', name='predictions')(x)

        self.model = keras.Model(inputs, outputs)

        # Compile model
        self.model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return self.model

    def create_data_generators(self, train_dir, validation_split=0.2, batch_size=32):
        """Create data generators for training"""

        # Data augmentation for training
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=validation_split
        )

        # Generator for validation (no augmentation)
        val_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=validation_split
        )

        # Training generator
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=self.input_shape[:2],
            batch_size=batch_size,
            class_mode='sparse',
            subset='training',
            shuffle=True
        )

        # Validation generator
        val_generator = val_datagen.flow_from_directory(
            train_dir,
            target_size=self.input_shape[:2],
            batch_size=batch_size,
            class_mode='sparse',
            subset='validation',
            shuffle=False
        )

        return train_generator, val_generator

    def train_model(self, train_generator, val_generator, epochs=20):
        """Train the model"""

        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=3,
                min_lr=1e-7
            ),
            keras.callbacks.ModelCheckpoint(
                'models/best_uniform_model.h5',
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
        ]

        # Train model
        print("Starting training...")
        self.history = self.model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // train_generator.batch_size,
            validation_data=val_generator,
            validation_steps=val_generator.samples // val_generator.batch_size,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

        print("Training completed!")
        return self.history

    def fine_tune_model(self, train_generator, val_generator, epochs=10):
        """Fine-tune the model by unfreezing some layers"""

        # Unfreeze the base model
        self.model.layers[2].trainable = True  # EfficientNet base model

        # Use lower learning rate for fine-tuning
        self.model.compile(
            optimizer=keras.optimizers.Adam(1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        print("Fine-tuning model...")
        fine_tune_history = self.model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // train_generator.batch_size,
            validation_data=val_generator,
            validation_steps=val_generator.samples // val_generator.batch_size,
            epochs=epochs,
            callbacks=[
                keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
                keras.callbacks.ModelCheckpoint(
                    'models/best_uniform_model_finetuned.h5',
                    monitor='val_accuracy',
                    save_best_only=True,
                    mode='max'
                )
            ],
            verbose=1
        )

        return fine_tune_history

    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("No training history available")
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

        # Plot accuracy
        ax1.plot(self.history.history['accuracy'], label='Training Accuracy')
        ax1.plot(self.history.history['val_accuracy'], label='Validation Accuracy')
        ax1.set_title('Model Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()
        ax1.grid(True)

        # Plot loss
        ax2.plot(self.history.history['loss'], label='Training Loss')
        ax2.plot(self.history.history['val_loss'], label='Validation Loss')
        ax2.set_title('Model Loss')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        plt.savefig('results/training_history.png', dpi=300, bbox_inches='tight')
        plt.show()

    def evaluate_model(self, test_generator):
        """Evaluate model performance"""

        # Get predictions
        test_generator.reset()
        predictions = self.model.predict(test_generator, verbose=1)
        predicted_classes = np.argmax(predictions, axis=1)

        # Get true labels
        true_classes = test_generator.classes
        class_labels = list(test_generator.class_indices.keys())

        # Print classification report
        print("Classification Report:")
        print(classification_report(true_classes, predicted_classes, target_names=class_labels))

        # Plot confusion matrix
        cm = confusion_matrix(true_classes, predicted_classes)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig('results/confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.show()

        return predictions, predicted_classes

    def predict_single_image(self, image_path):
        """Predict single image"""

        # Load and preprocess image
        img = keras.utils.load_img(image_path, target_size=self.input_shape[:2])
        img_array = keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

        # Make prediction
        predictions = self.model.predict(img_array)
        predicted_class = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class]

        return predicted_class, confidence

    def save_model(self, filepath='models/uniform_classifier.h5'):
        """Save the trained model"""
        self.model.save(filepath)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath='models/uniform_classifier.h5'):
        """Load a trained model"""
        self.model = keras.models.load_model(filepath)
        print(f"Model loaded from {filepath}")

# Usage example
def train_uniform_classifier():
    """Complete training pipeline"""

    # Initialize classifier
    classifier = UniformClassifier()

    # Create model
    model = classifier.create_model()
    print("Model created!")
    print(f"Total parameters: {model.count_params():,}")

    # Check if organized dataset exists
    train_dir = "organized_dataset/train"
    if not os.path.exists(train_dir):
        print("Organized dataset not found!")
        print("Please create the directory structure:")
        print("organized_dataset/")
        print("├── train/")
        print("│   ├── uniform/")
        print("│   └── non_uniform/")
        print("└── test/")
        print("    ├── uniform/")
        print("    └── non_uniform/")
        return None

    # Create data generators
    train_gen, val_gen = classifier.create_data_generators(train_dir)

    print(f"Training samples: {train_gen.samples}")
    print(f"Validation samples: {val_gen.samples}")
    print(f"Classes: {train_gen.class_indices}")

    # Train model
    history = classifier.train_model(train_gen, val_gen, epochs=20)

    # Plot training history
    classifier.plot_training_history()

    # Fine-tune model
    fine_tune_history = classifier.fine_tune_model(train_gen, val_gen, epochs=10)

    # Save model
    classifier.save_model()

    return classifier

# Initialize
print("Uniform classifier module ready!")
print("Make sure to organize your data first, then run train_uniform_classifier()")

Uniform classifier module ready!
Make sure to organize your data first, then run train_uniform_classifier()


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
import json
from datetime import datetime

class UniformFraudDetector:
    def __init__(self, model_path='models/uniform_classifier.h5'):
        """Initialize the complete fraud detection system"""

        # Initialize components
        from image_segmentation import UniformSegmenter
        from classification_model import UniformClassifier

        self.segmenter = UniformSegmenter()
        self.classifier = UniformClassifier()

        # Load trained model
        if os.path.exists(model_path):
            self.classifier.load_model(model_path)
            print("Trained model loaded successfully!")
        else:
            print(f"Model not found at {model_path}. Please train the model first.")

    def detect_fraud(self, image_path, confidence_threshold=0.7):
        """Complete fraud detection pipeline"""

        results = {
            'image_path': image_path,
            'timestamp': datetime.now().isoformat(),
            'fraud_detected': False,
            'confidence_scores': {},
            'regions_analyzed': [],
            'overall_confidence': 0.0,
            'verdict': 'GENUINE'
        }

        try:
            # Step 1: Load and preprocess image
            original_image = cv2.imread(image_path)
            if original_image is None:
                results['error'] = "Could not load image"
                return results

            # Step 2: Segment person and extract regions
            segmented_image, mask, regions = self.segmenter.process_image(image_path)

            if not regions:
                results['error'] = "No person detected in image"
                return results

            # Step 3: Classify each extracted region
            region_predictions = []
            total_confidence = 0

            for region_name, region_img, bbox in regions:
                # Save region as temporary file for classification
                temp_path = f"temp_{region_name}.jpg"
                cv2.imwrite(temp_path, region_img)

                # Classify region
                try:
                    predicted_class, confidence = self.classifier.predict_single_image(temp_path)

                    region_result = {
                        'region': region_name,
                        'predicted_class': int(predicted_class),
                        'confidence': float(confidence),
                        'is_uniform': predicted_class == 1,
                        'bbox': bbox
                    }

                    region_predictions.append(region_result)
                    results['regions_analyzed'].append(region_result)
                    results['confidence_scores'][region_name] = float(confidence)

                    total_confidence += confidence

                    # Clean up temp file
                    if os.path.exists(temp_path):
                        os.remove(temp_path)

                except Exception as e:
                    print(f"Error classifying {region_name}: {e}")
                    continue

            # Step 4: Make overall decision
            if region_predictions:
                # Calculate overall confidence
                results['overall_confidence'] = total_confidence / len(region_predictions)

                # Check if any region is classified as uniform with high confidence
                uniform_detections = [r for r in region_predictions if r['is_uniform'] and r['confidence'] > confidence_threshold]

                if uniform_detections:
                    results['fraud_detected'] = True
                    results['verdict'] = 'POTENTIAL FRAUD - UNIFORM DETECTED'

                    # Find the most confident uniform detection
                    best_detection = max(uniform_detections, key=lambda x: x['confidence'])
                    results['primary_detection'] = best_detection
                else:
                    results['verdict'] = 'GENUINE - NO UNIFORM DETECTED'

        except Exception as e:
            results['error'] = str(e)
            print(f"Error in fraud detection: {e}")

        return results

    def visualize_detection_results(self, image_path, results):
        """Visualize detection results"""

        # Load original image
        original_image = cv2.imread(image_path)
        if original_image is None:
            return

        # Convert to RGB for matplotlib
        display_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        # Create figure
        fig, axes = plt.subplots(1, 2, figsize=(15, 8))

        # Show original image
        axes[0].imshow(display_image)
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        # Show image with detection boxes
        result_image = display_image.copy()

        if 'regions_analyzed' in results:
            for region in results['regions_analyzed']:
                bbox = region['bbox']
                x1, y1, x2, y2 = bbox

                # Choose color based on classification
                color = (255, 0, 0) if region['is_uniform'] else (0, 255, 0)  # Red for uniform, Green for non-uniform
                thickness = 3 if region['is_uniform'] else 2

                # Draw rectangle
                cv2.rectangle(result_image, (x1, y1), (x2, y2), color, thickness)

                # Add label
                label = f"{region['region']}: {'UNIFORM' if region['is_uniform'] else 'CIVILIAN'} ({region['confidence']:.2f})"
                cv2.putText(result_image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        axes[1].imshow(result_image)
        axes[1].set_title(f"Detection Results - {results['verdict']}")
        axes[1].axis('off')

        # Add results text
        results_text = f"Overall Confidence: {results['overall_confidence']:.2f}\n"
        results_text += f"Fraud Detected: {results['fraud_detected']}\n"
        results_text += f"Regions Analyzed: {len(results.get('regions_analyzed', []))}"

        fig.suptitle(results_text, fontsize=12)
        plt.tight_layout()

        # Save results
        output_path = f"results/detection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.show()

        return output_path

    def batch_process_images(self, image_folder, output_file='results/batch_results.json'):
        """Process multiple images and save results"""

        all_results = []
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

        # Get all image files
        image_files = [f for f in os.listdir(image_folder)
                      if any(f.lower().endswith(ext) for ext in image_extensions)]

        print(f"Processing {len(image_files)} images...")

        for i, filename in enumerate(image_files):
            print(f"Processing {i+1}/{len(image_files)}: {filename}")

            image_path = os.path.join(image_folder, filename)
            results = self.detect_fraud(image_path)
            all_results.append(results)

            # Visualize results for first few images
            if i < 5:  # Show first 5 results
                self.visualize_detection_results(image_path, results)

        # Save all results
        with open(output_file, 'w') as f:
            json.dump(all_results, f, indent=2)

        # Generate summary statistics
        self.generate_batch_summary(all_results)

        return all_results

    def generate_batch_summary(self, results):
        """Generate summary statistics from batch processing"""

        if not results:
            return

        total_images = len(results)
        fraud_detected = sum(1 for r in results if r.get('fraud_detected', False))
        successful_processing = sum(1 for r in results if 'error' not in r)

        # Calculate average confidence
        confidences = [r['overall_confidence'] for r in results if r.get('overall_confidence', 0) > 0]
        avg_confidence = sum(confidences) / len(confidences) if confidences else 0

        print("\n" + "="*50)
        print("BATCH PROCESSING SUMMARY")
        print("="*50)
        print(f"Total Images Processed: {total_images}")
        print(f"Successfully Processed: {successful_processing}")
        print(f"Potential Fraud Detected: {fraud_detected} ({fraud_detected/total_images*100:.1f}%)")
        print(f"Average Confidence Score: {avg_confidence:.2f}")
        print(f"Processing Success Rate: {successful_processing/total_images*100:.1f}%")

        # Region analysis summary
        region_counts = {}
        for result in results:
            for region in result.get('regions_analyzed', []):
                region_name = region['region']
                region_counts[region_name] = region_counts.get(region_name, 0) + 1

        if region_counts:
            print(f"\nRegions Analyzed:")
            for region, count in region_counts.items():
                print(f"  {region}: {count} detections")

        print("="*50)

# Testing and demonstration functions
def test_single_image(image_path):
    """Test fraud detection on a single image"""

    detector = UniformFraudDetector()
    results = detector.detect_fraud(image_path)
    detector.visualize_detection_results(image_path, results)

    print("\nDetection Results:")
    print(f"Verdict: {results['verdict']}")
    print(f"Fraud Detected: {results['fraud_detected']}")
    print(f"Overall Confidence: {results['overall_confidence']:.2f}")

    if results.get('regions_analyzed'):
        print("\nRegion Analysis:")
        for region in results['regions_analyzed']:
            print(f"  {region['region']}: {'UNIFORM' if region['is_uniform'] else 'CIVILIAN'} "
                  f"(confidence: {region['confidence']:.2f})")

    return results

def run_demo():
    """Run a complete demonstration"""

    print("UNIFORM FRAUD DETECTION SYSTEM DEMO")
    print("="*50)

    # Check if model exists
    if not os.path.exists('models/uniform_classifier.h5'):
        print("❌ Trained model not found!")
        print("Please train the model first using the classification_model.py script")
        return

    # Check if test images exist
    test_folder = "Uniform/test"
    if not os.path.exists(test_folder):
        print("❌ Test folder not found!")
        print("Please upload test images to Uniform/test/ folder")
        return

    # Initialize detector
    try:
        detector = UniformFraudDetector()
        print("✅ Fraud detector initialized successfully!")

        # Process test images
        print("\nProcessing test images...")
        results = detector.batch_process_images(test_folder)

        print(f"✅ Processed {len(results)} images successfully!")

    except Exception as e:
        print(f"❌ Error during demo: {e}")

# Main execution
if __name__ == "__main__":
    print("Complete Uniform Fraud Detection Pipeline Ready!")
    print("\nAvailable functions:")
    print("- test_single_image(image_path): Test on single image")
    print("- run_demo(): Run complete demonstration")
    print("- UniformFraudDetector(): Initialize detector class")

    # Uncomment to run demo automatically
    # run_demo()

Complete Uniform Fraud Detection Pipeline Ready!

Available functions:
- test_single_image(image_path): Test on single image
- run_demo(): Run complete demonstration
- UniformFraudDetector(): Initialize detector class


In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from complete_pipeline import UniformFraudDetector # Remove this line
from __main__ import UniformFraudDetector # Import from the current notebook environment

class ModelEvaluator:
    def __init__(self, model_path='models/uniform_classifier.h5'):
        self.detector = UniformFraudDetector(model_path)
        self.test_results = []

    def evaluate_test_dataset(self, test_folder, ground_truth_file=None):
        """Evaluate model on test dataset with known labels"""

        # Create ground truth mapping if not provided
        if ground_truth_file is None:
            ground_truth = self.create_ground_truth_mapping(test_folder)
        else:
            with open(ground_truth_file, 'r') as f:
                ground_truth = json.load(f)

        print(f"Evaluating on {len(ground_truth)} test images...")

        predictions = []
        true_labels = []
        detailed_results = []

        for filename, true_label in ground_truth.items():
            image_path = os.path.join(test_folder, filename)

            if not os.path.exists(image_path):
                continue

            print(f"Testing: {filename}")

            # Run fraud detection
            result = self.detector.detect_fraud(image_path)

            # Extract prediction
            predicted_label = 1 if result['fraud_detected'] else 0

            predictions.append(predicted_label)
            true_labels.append(true_label)

            # Store detailed results
            detailed_results.append({
                'filename': filename,
                'true_label': true_label,
                'predicted_label': predicted_label,
                'confidence': result['overall_confidence'],
                'verdict': result['verdict'],
                'regions_detected': len(result.get('regions_analyzed', []))
            })

        # Calculate metrics
        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions)
        recall = recall_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions)

        metrics = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'total_samples': len(true_labels),
            'true_positives': sum(1 for t, p in zip(true_labels, predictions) if t == 1 and p == 1),
            'false_positives': sum(1 for t, p in zip(true_labels, predictions) if t == 0 and p == 1),
            'true_negatives': sum(1 for t, p in zip(true_labels, predictions) if t == 0 and p == 0),
            'false_negatives': sum(1 for t, p in zip(true_labels, predictions) if t == 1 and p == 0)
        }

        # Save detailed results
        self.save_evaluation_results(detailed_results, metrics)

        # Create visualizations
        self.create_evaluation_plots(true_labels, predictions, detailed_results)

        return metrics, detailed_results

    def create_ground_truth_mapping(self, test_folder):
        """Create ground truth mapping based on folder structure or filename patterns"""

        ground_truth = {}

        # Check if organized structure exists
        uniform_folder = os.path.join(test_folder, 'uniform')
        non_uniform_folder = os.path.join(test_folder, 'non_uniform')

        if os.path.exists(uniform_folder) and os.path.exists(non_uniform_folder):
            # Organized structure
            for filename in os.listdir(uniform_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    ground_truth[filename] = 1  # Uniform

            for filename in os.listdir(non_uniform_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    ground_truth[filename] = 0  # Non-uniform
        else:
            # Fallback: use filename patterns
            for filename in os.listdir(test_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    # Simple pattern matching - you may need to adjust this
                    if any(keyword in filename.lower() for keyword in ['police', 'officer', 'uniform', 'army', 'military']):
                        ground_truth[filename] = 1
                    else:
                        ground_truth[filename] = 0

        print(f"Created ground truth mapping for {len(ground_truth)} images")
        print(f"Uniform samples: {sum(ground_truth.values())}")
        print(f"Non-uniform samples: {len(ground_truth) - sum(ground_truth.values())}")

        return ground_truth

    def save_evaluation_results(self, detailed_results, metrics):
        """Save evaluation results to files"""

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Save detailed results
        with open(f'results/detailed_results_{timestamp}.json', 'w') as f:
            json.dump(detailed_results, f, indent=2)

        # Save metrics
        with open(f'results/metrics_{timestamp}.json', 'w') as f:
            json.dump(metrics, f, indent=2)

        # Create CSV for easy analysis
        df = pd.DataFrame(detailed_results)
        df.to_csv(f'results/results_{timestamp}.csv', index=False)

        print(f"Results saved with timestamp: {timestamp}")

    def create_evaluation_plots(self, true_labels, predictions, detailed_results):
        """Create evaluation visualizations"""

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))

        # Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(true_labels, predictions)

        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Non-Uniform', 'Uniform'],
                   yticklabels=['Non-Uniform', 'Uniform'], ax=axes[0,0])
        axes[0,0].set_title('Confusion Matrix')
        axes[0,0].set_xlabel('Predicted')
        axes[0,0].set_ylabel('Actual')

        # Confidence Distribution
        df = pd.DataFrame(detailed_results)
        df_correct = df[df['true_label'] == df['predicted_label']]
        df_incorrect = df[df['true_label'] != df['predicted_label']]

        axes[0,1].hist(df_correct['confidence'], alpha=0.7, label='Correct Predictions', bins=20)
        axes[0,1].hist(df_incorrect['confidence'], alpha=0.7, label='Incorrect Predictions', bins=20)
        axes[0,1].set_title('Confidence Score Distribution')
        axes[0,1].set_xlabel('Confidence Score')
        axes[0,1].set_ylabel('Frequency')
        axes[0,1].legend()

        # Accuracy by Confidence Threshold
        thresholds = np.arange(0.1, 1.0, 0.05)
        accuracies = []

        for threshold in thresholds:
            # Filter predictions by confidence threshold
            high_conf_mask = df['confidence'] >= threshold
            if high_conf_mask.sum() > 0:
                filtered_true = df[high_conf_mask]['true_label']
                filtered_pred = df[high_conf_mask]['predicted_label']
                acc = accuracy_score(filtered_true, filtered_pred)
                accuracies.append(acc)
            else:
                accuracies.append(0)

        axes[1,0].plot(thresholds, accuracies, marker='o')
        axes[1,0].set_title('Accuracy vs Confidence Threshold')
        axes[1,0].set_xlabel('Minimum Confidence Threshold')
        axes[1,0].set_ylabel('Accuracy')
        axes[1,0].grid(True)

        # Sample Coverage by Confidence Threshold
        coverage = []
        for threshold in thresholds:
            coverage.append((df['confidence'] >= threshold).mean())

        axes[1,1].plot(thresholds, coverage, marker='s', color='orange')
        axes[1,1].set_title('Sample Coverage vs Confidence Threshold')
        axes[1,1].set_xlabel('Minimum Confidence Threshold')
        axes[1,1].set_ylabel('Fraction of Samples Retained')
        axes[1,1].grid(True)

        plt.tight_layout()
        plt.savefig('results/evaluation_plots.png', dpi=300, bbox_inches='tight')
        plt.show()

    def performance_report(self, metrics):
        """Print detailed performance report"""

        print("\n" + "="*60)
        print("MODEL PERFORMANCE REPORT")
        print("="*60)
        print(f"Dataset Size: {metrics['total_samples']} images")
        print(f"Accuracy: {metrics['accuracy']:.3f} ({metrics['accuracy']*100:.1f}%)")
        print(f"Precision: {metrics['precision']:.3f}")
        print(f"Recall: {metrics['recall']:.3f}")
        print(f"F1-Score: {metrics['f1_score']:.3f}")
        print("\nConfusion Matrix:")
        print(f"True Positives (Correctly detected fraud): {metrics['true_positives']}")
        print(f"True Negatives (Correctly detected genuine): {metrics['true_negatives']}")
        print(f"False Positives (Incorrectly flagged as fraud): {metrics['false_positives']}")
        print(f"False Negatives (Missed fraud cases): {metrics['false_negatives']}")

        # Calculate additional metrics
        specificity = metrics['true_negatives'] / (metrics['true_negatives'] + metrics['false_positives'])
        print(f"\nSpecificity (True Negative Rate): {specificity:.3f}")

        if metrics['true_positives'] + metrics['false_negatives'] > 0:
            sensitivity = metrics['true_positives'] / (metrics['true_positives'] + metrics['false_negatives'])
            print(f"Sensitivity (True Positive Rate): {sensitivity:.3f}")

        print("="*60)

def run_comprehensive_evaluation():
    """Run comprehensive evaluation of the fraud detection system"""

    print("Starting Comprehensive Model Evaluation...")

    # Check if model exists
    if not os.path.exists('models/uniform_classifier.h5'):
        print("❌ Trained model not found!")
        print("Please train the model first.")
        return

    # Initialize evaluator
    evaluator = ModelEvaluator()

    # Check test data structure
    test_folder = "organized_dataset/test"
    if not os.path.exists(test_folder):
        test_folder = "Uniform/test"
        if not os.path.exists(test_folder):
            print("❌ Test folder not found!")
            print("Please ensure test data is available in 'organized_dataset/test' or 'Uniform/test'")
            return

    try:
        # Run evaluation
        print(f"Using test folder: {test_folder}")
        metrics, detailed_results = evaluator.evaluate_test_dataset(test_folder)

        # Print performance report
        evaluator.performance_report(metrics)

        # Analyze failure cases
        df = pd.DataFrame(detailed_results)
        failures = df[df['true_label'] != df['predicted_label']]

        if len(failures) > 0:
            print(f"\nFAILURE ANALYSIS:")
            print(f"Total failures: {len(failures)}")
            print("Failed cases:")
            for _, row in failures.head(10).iterrows():  # Show first 10 failures
                print(f"  {row['filename']}: True={row['true_label']}, "
                      f"Pred={row['predicted_label']}, Conf={row['confidence']:.2f}")

        print("\n✅ Evaluation completed successfully!")

    except Exception as e:
        print(f"❌ Error during evaluation: {e}")
        import traceback
        traceback.print_exc()

# Quick test functions
def quick_test(image_path):
    """Quick test on a single image"""

    detector = UniformFraudDetector()
    result = detector.detect_fraud(image_path)
    detector.visualize_detection_results(image_path, result)

    print(f"\nQuick Test Results for: {os.path.basename(image_path)}")
    print(f"Verdict: {result['verdict']}")
    print(f"Confidence: {result['overall_confidence']:.2f}")

    return result

if __name__ == "__main__":
    print("Model Evaluation and Testing Module Ready!")
    print("\nAvailable functions:")
    print("- run_comprehensive_evaluation(): Full model evaluation")
    print("- quick_test(image_path): Quick test on single image")

    # Uncomment to run evaluation automatically
    # run_comprehensive_evaluation()

Model Evaluation and Testing Module Ready!

Available functions:
- run_comprehensive_evaluation(): Full model evaluation
- quick_test(image_path): Quick test on single image


In [ ]:
import os
import sys
import shutil
from datetime import datetime

def setup_project_structure():
    """Create project directory structure"""

    directories = [
        'models',
        'results',
        'data',
        'organized_dataset/train/uniform',
        'organized_dataset/train/non_uniform',
        'organized_dataset/test/uniform',
        'organized_dataset/test/non_uniform',
        'temp'
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)

    print("✅ Project structure created!")

def organize_existing_data():
    """Help organize existing data into proper structure"""

    # Check if original Uniform folder exists
    if os.path.exists('Uniform'):
        print("Found existing 'Uniform' folder with data.")
        print("Please manually organize images into:")
        print("📁 organized_dataset/")
        print("  ├── 📁 train/")
        print("  │   ├── 📁 uniform/ (put uniform images here)")
        print("  │   └── 📁 non_uniform/ (put civilian clothes images here)")
        print("  └── 📁 test/")
        print("      ├── 📁 uniform/ (put test uniform images here)")
        print("      └── 📁 non_uniform/ (put test civilian images here)")
        print()
        print("💡 Tip: Aim for 200-300 images per category for best results")

        # Create example organization script
        create_organization_helper()
    else:
        print("No existing 'Uniform' folder found.")
        print("Please upload your images and organize them as described above.")

def create_organization_helper():
    """Create a helper script to organize data"""

    helper_code = '''
# Data Organization Helper Script
import os
import shutil
import random

def organize_images():
    """Helper to organize images - modify as needed"""

    source_train = "Uniform/train"
    source_test = "Uniform/test"

    if os.path.exists(source_train):
        print(f"Found {len(os.listdir(source_train))} images in train folder")

        # You'll need to manually sort these based on content
        for filename in os.listdir(source_train):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                print(f"Process: {filename}")
                # Manually move to appropriate folder:
                # shutil.copy(os.path.join(source_train, filename),
                #            "organized_dataset/train/uniform/")  # or non_uniform

    print("Manual organization required - examine each image and sort accordingly")

if __name__ == "__main__":
    organize_images()
'''

    with open('organize_data.py', 'w') as f:
        f.write(helper_code)

    print("Created 'organize_data.py' helper script")

def check_data_ready():
    """Check if data is properly organized and ready"""

    train_uniform = "organized_dataset/train/uniform"
    train_non_uniform = "organized_dataset/train/non_uniform"
    test_uniform = "organized_dataset/test/uniform"
    test_non_uniform = "organized_dataset/test/non_uniform"

    counts = {}
    for folder_name, folder_path in [
        ("Train Uniform", train_uniform),
        ("Train Non-Uniform", train_non_uniform),
        ("Test Uniform", test_uniform),
        ("Test Non-Uniform", test_non_uniform)
    ]:
        if os.path.exists(folder_path):
            count = len([f for f in os.listdir(folder_path)
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            counts[folder_name] = count
        else:
            counts[folder_name] = 0

    print("📊 Data Organization Status:")
    for category, count in counts.items():
        status = "✅" if count > 50 else "⚠️" if count > 10 else "❌"
        print(f"{status} {category}: {count} images")

    total_images = sum(counts.values())
    min_recommended = 800  # 200 per category

    if total_images >= min_recommended:
        print(f"\n✅ Dataset ready! Total: {total_images} images")
        return True
    else:
        print(f"\n⚠️ Dataset needs more images. Total: {total_images}, Recommended: {min_recommended}+")
        print("Consider data augmentation or collecting more images")
        return total_images > 100  # Minimum viable dataset

def train_complete_model():
    """Train the complete fraud detection model"""

    print("\n🚀 Starting Model Training Pipeline...")

    try:
        # Import training modules
        from classification_model import train_uniform_classifier

        print("Step 1: Training EfficientNet-based classifier...")
        classifier = train_uniform_classifier()

        if classifier is not None:
            print("✅ Model training completed successfully!")
            print("Model saved to: models/uniform_classifier.h5")
            return True
        else:
            print("❌ Model training failed!")
            return False

    except Exception as e:
        print(f"❌ Error during training: {e}")
        import traceback
        traceback.print_exc()
        return False

def run_demo_evaluation():
    """Run demonstration and evaluation"""

    print("\n🧪 Running Model Evaluation...")

    try:
        from testing_evaluation import run_comprehensive_evaluation
        run_comprehensive_evaluation()

    except Exception as e:
        print(f"❌ Error during evaluation: {e}")
        import traceback
        traceback.print_exc()

def create_demo_video_script():
    """Create script for recording demo video"""

    demo_script = '''
# Demo Video Recording Script
# Use this as a guide for creating your demo video

"""
DEMO VIDEO SCRIPT - Uniform Fraud Detection System

1. INTRODUCTION (30 seconds)
   - "This is a machine learning system to detect fraud calls where criminals impersonate law enforcement"
   - Show project structure and files

2. DATA ORGANIZATION (60 seconds)
   - Show organized_dataset folder structure
   - Explain uniform vs non_uniform categories
   - Display sample images from each category

3. MODEL TRAINING (90 seconds)
   - Run: python main.py --train
   - Show training progress and metrics
   - Explain EfficientNet-Lite4 architecture

4. SEGMENTATION DEMO (60 seconds)
   - Show person segmentation using MediaPipe
   - Demonstrate region extraction (head, torso)
   - Visualize segmentation results

5. CLASSIFICATION DEMO (90 seconds)
   - Test on sample images
   - Show confidence scores and predictions
   - Demonstrate both genuine and fraud cases

6. COMPLETE PIPELINE (120 seconds)
   - Run: python main.py --test <image_path>
   - Run batch processing: python main.py --batch <folder_path>
   - Show batch results and evaluation report
   - Discuss system performance and potential improvements

7. CONCLUSION (30 seconds)
   - Summarize key features and benefits
   - Briefly mention future work or next steps
   - Thank viewers and provide contact info (optional)
"""
''' # Added the closing triple quotes here

    with open('demo_script.txt', 'w') as f:
        f.write(demo_script)

    print("Created 'demo_script.txt' for demo video recording")

# Main execution
if __name__ == "__main__":
    print("Complete Uniform Fraud Detection Pipeline Setup Ready!")
    print("\nAvailable functions:")
    print("- setup_project_structure(): Create project directory structure")
    print("- organize_existing_data(): Help organize existing data")
    print("- check_data_ready(): Check data organization status")
    print("- train_complete_model(): Train the model")
    print("- run_demo_evaluation(): Run demo and evaluation")
    print("- create_demo_video_script(): Create script for demo video")

    # Uncomment to run initial setup
    # setup_project_structure()
    # organize_existing_data()

    # Uncomment to check data status
    # check_data_ready()

    # Uncomment to train model
    # train_complete_model()

    # Uncomment to run evaluation
    # run_demo_evaluation()

    # Uncomment to create demo script
    # create_demo_video_script()

Complete Uniform Fraud Detection Pipeline Setup Ready!

Available functions:
- setup_project_structure(): Create project directory structure
- organize_existing_data(): Help organize existing data
- check_data_ready(): Check data organization status
- train_complete_model(): Train the model
- run_demo_evaluation(): Run demo and evaluation
- create_demo_video_script(): Create script for demo video
